In [ ]:
# default_exp diff_analysis_manager

In [ ]:
from ms_empire.background_distributions import *
from ms_empire.normalization import *
from ms_empire.diff_analysis import *

In [ ]:
#export
#read in peptide datafiles



In [ ]:
#export
import pandas as pd
from itertools import combinations
import numpy as np

def run_pipeline(unnormed_df, labelmap_df, minrep):
    conds = labelmap_df["condition"].unique()
    condpair2diffions = {}
    for condpair in combinations(conds, 2):
        
        c1_samples = labelmap_df[labelmap_df["condition"]== condpair[0]]
        c2_samples = labelmap_df[labelmap_df["condition"]== condpair[1]]
        df_c1 = unnormed_df.loc[:, c1_samples["sample"]].dropna(thresh=c1_samples.shape[0]-minrep, axis=0)
        df_c2 = unnormed_df.loc[:, c2_samples["sample"]].dropna(thresh=c2_samples.shape[0]-minrep, axis=0)
        display(df_c1)
        df_c1_normed = pd.DataFrame(normalize_withincond(df_c1.to_numpy().T).T, index = df_c1.index)
        print("end norm1")
        display(df_c1_normed)
        df_c2_normed = pd.DataFrame(normalize_withincond(df_c2.to_numpy().T).T, index = df_c2.index)
        
        normed_c1 = ConditionBackgrounds(df_c1_normed)
        normed_c2 = ConditionBackgrounds(df_c2_normed)

        ions_to_check = normed_c1.ion2nonNanvals.keys() & normed_c2.ion2nonNanvals.keys()

        ion2diffDist = {}
        diffIons = []
        for ion in ions_to_check:
            vals1 = normed_c1.ion2nonNanvals.get(ion)
            vals2 = normed_c2.ion2nonNanvals.get(ion)
            bgdist1 = normed_c1.ion2background.get(ion)
            bgdist2 = normed_c2.ion2background.get(ion)
            diffDist = get_subtracted_bg(ion2diffDist,bgdist1, bgdist2,ion )
            diffIon = DifferentialIon(vals1, vals2, diffDist)
            diffIons.append(diffIon)
        
        condpair2diffions.update({condpair :diffIon})

In [ ]:
#export
import pandas as pd
from itertools import combinations
import numpy as np

def run_pipeline(unnormed_df, labelmap_df, minrep):
    conds = labelmap_df["condition"].unique()
    condpair2diffions = {}
    for condpair in combinations(conds, 2):

        c1_samples = labelmap_df[labelmap_df["condition"]== condpair[0]]
        c2_samples = labelmap_df[labelmap_df["condition"]== condpair[1]]
        df_c1 = unnormed_df.loc[:, c1_samples["sample"]].dropna(thresh=c1_samples.shape[0]-minrep, axis=0)
        df_c2 = unnormed_df.loc[:, c2_samples["sample"]].dropna(thresh=c2_samples.shape[0]-minrep, axis=0)
        
        df_c1_normed = pd.DataFrame(normalize_withincond(df_c1.to_numpy().T).T, index = df_c1.index)
        df_c2_normed = pd.DataFrame(normalize_withincond(df_c2.to_numpy().T).T, index = df_c2.index)

        normed_c1 = ConditionBackgrounds(df_c1_normed)
        normed_c2 = ConditionBackgrounds(df_c2_normed)

        ions_to_check = normed_c1.ion2nonNanvals.keys() & normed_c2.ion2nonNanvals.keys()

        ion2diffDist = {}
        diffIons = []
        for ion in ions_to_check:
            vals1 = normed_c1.ion2nonNanvals.get(ion)
            vals2 = normed_c2.ion2nonNanvals.get(ion)
            bgdist1 = normed_c1.ion2background.get(ion)
            bgdist2 = normed_c2.ion2background.get(ion)
            diffDist = get_subtracted_bg(ion2diffDist,bgdist1, bgdist2,ion )
            diffIon = DifferentialIon(vals1, vals2, diffDist)
            diffIons.append(diffIon)

        condpair2diffions.update({condpair :diffIon})

In [ ]:
#hide
import pandas as pd
import numpy as np
import uuid

def test_run_pipeline():

    sample2cond_df = pd.DataFrame({'sample' : ['A1', 'A2', 'A3', 'B1', 'B2', 'B3'], 
    'condition' : ['A', 'A', 'A', 'B', 'B', 'B']})
    unnormed_df = generate_random_input(10000, sample2cond_df)
    
    run_pipeline(unnormed_df, sample2cond_df, 3)


def generate_random_input(num_pep,sample2cond_df ):
    pepnames = list(map(lambda _idx : str(uuid.uuid4()), range(num_pep))) #gives uuid strings for each peptide
    randarrays = 10+ 1.5*np.random.randn(num_pep,sample2cond_df.shape[0])
    df_intens = pd.DataFrame(randarrays, columns= sample2cond_df["sample"].tolist())
    df_intens.insert(0, "peptides", pepnames )
    df_intens = df_intens.set_index("peptides")
    return df_intens

test_run_pipeline()


,A1,A2,A3
peptides,,,
6a3e6b81-148b-4913-808b-8e91bd0f7d3e,11.386111,10.300777,9.594174
43e99dea-718b-4a8f-8ff8-789a3f53374d,11.275167,9.948974,11.721857
aa5f90e6-aa0a-4a88-a0ab-2c741ec1169b,10.903977,9.681779,9.852887
270145c3-c4a9-4a22-842d-4d726e037827,8.820409,9.932944,8.083470
c5fa5d7b-5d0d-4ef7-80ea-467d6a649568,8.440354,9.122972,10.338375
...,...,...,...
65950ccc-fd2f-4af2-b111-1ca9eda6606b,11.509924,10.378830,10.590148
9c00cf26-71c0-4a95-ab50-195fcc842561,8.994162,10.551511,11.107499
21c04778-ce1b-4d6f-a0bb-08fa77135050,9.868574,11.749465,9.654644


end norm1


,0,1,2
peptides,,,
6a3e6b81-148b-4913-808b-8e91bd0f7d3e,11.386111,10.291599,9.560770
43e99dea-718b-4a8f-8ff8-789a3f53374d,11.275167,9.939796,11.688453
aa5f90e6-aa0a-4a88-a0ab-2c741ec1169b,10.903977,9.672601,9.819483
270145c3-c4a9-4a22-842d-4d726e037827,8.820409,9.923766,8.050065
c5fa5d7b-5d0d-4ef7-80ea-467d6a649568,8.440354,9.113795,10.304971
...,...,...,...
65950ccc-fd2f-4af2-b111-1ca9eda6606b,11.509924,10.369653,10.556744
9c00cf26-71c0-4a95-ab50-195fcc842561,8.994162,10.542333,11.074095
21c04778-ce1b-4d6f-a0bb-08fa77135050,9.868574,11.740287,9.621240


NameError: name 'assign_ions2bgdists' is not defined